In [43]:
from __future__ import print_function
import pandas as pd


from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, LSTM, Dense
import numpy as np

In [44]:
train = "training_data_orig.csv"
filings = "extract_data_vendor.tsv"
docs = 100000

In [45]:
dft = pd.read_csv(train, nrows = docs)
dft.head()

,slug,page,x0,y0,x1,y1,token,gross_amount
0,459790-nrsc_mandel_rep_senate_1010_101412_314271,0.0,423.000,25.145,439.448,25.189,Print,0.00
1,459790-nrsc_mandel_rep_senate_1010_101412_314271,0.0,441.672,25.145,458.568,25.189,Date,0.00
2,459790-nrsc_mandel_rep_senate_1010_101412_314271,0.0,474.001,25.145,505.137,25.189,10/9/12,0.40
3,459790-nrsc_mandel_rep_senate_1010_101412_314271,0.0,533.251,25.145,551.931,25.189,Page,0.00
4,459790-nrsc_mandel_rep_senate_1010_101412_314271,0.0,560.251,25.145,564.699,25.189,1,0.25


In [46]:
dff = pd.read_csv(filings,nrows = docs, sep='\t')
dff.head()


,id,filing_type,contract_number,url,committee,agency,callsign,dc_slug,thumbnail_url,gross_amount,market_id,upload_date
0,23969,Federal,314271,/collect/files/59438/Political File/2012/Feder...,NRSC-MANDEL/REPUBLICAN,STRATEGIC MEDIA SERVICES,WCPO-TV,459790-nrsc_mandel_rep_senate_1010_101412_314271,https://s3.amazonaws.com/s3.documentcloud.org/...,21100.0,35,2012-10-10 00:00:00 UTC
1,2333,Non-Candidate Issue Ads,9792042,/collect/files/11289/Political File/2012/Non-C...,60+ ASSOCIATION,MENTZER MEDIA SERVICES,WKRC-TV,416491-collect-files-11289-political-file-2012...,https://s3.amazonaws.com/s3.documentcloud.org/...,6250.0,35,2012-08-14 00:00:00 UTC
2,31990,Federal,336298,/collect/files/73910/Political File/2012/Feder...,POL/R TERRY/D/PRES/US,KATHLEEN OFFERMAN,WPXI,470297-101612-terry-contract-2-13504017232906-...,https://s3.amazonaws.com/s3.documentcloud.org/...,30.0,23,2012-10-16 00:00:00 UTC
3,33211,Federal,116252,/collect/files/73123/Political File/2012/Feder...,COMMITTEE TO ELECT SHANELLE JACKSON,SHANELLE JACKSON FOR CONGRESS,WJBK,473630-116252-0-13442821773323-_-pdf,https://s3.amazonaws.com/s3.documentcloud.org/...,1170.0,11,2012-08-06 00:00:00 UTC
4,6213,Non-Candidate Issue Ads,48982,/collect/files/72313/Political File/2012/Non-C...,PRIORITIES USA ACTION,MUNDY KATOWITZ MEDIA,WHP-TV,420426-collect-files-72313-political-file-2012...,https://s3.amazonaws.com/s3.documentcloud.org/...,9335.0,41,2012-08-13 00:00:00 UTC


Useful Bash Code: 
    
    
    head -n 50 training.csv
    head -n 30 ftf-all-filings.tsv

In [47]:
df_all = pd.merge(left=dft, right=dff, how='left', left_on='slug', right_on='dc_slug')
#df_all = df_all[['slug', 'page', 'x0', 'y0', 'x1', 'y1', 'token', 'gross_amount_x', 'committee']]
df_all = df_all[['slug', 'token', 'committee']]

df_all.head(10)

,slug,token,committee
0,459790-nrsc_mandel_rep_senate_1010_101412_314271,Print,NRSC-MANDEL/REPUBLICAN
1,459790-nrsc_mandel_rep_senate_1010_101412_314271,Date,NRSC-MANDEL/REPUBLICAN
2,459790-nrsc_mandel_rep_senate_1010_101412_314271,10/9/12,NRSC-MANDEL/REPUBLICAN
3,459790-nrsc_mandel_rep_senate_1010_101412_314271,Page,NRSC-MANDEL/REPUBLICAN
4,459790-nrsc_mandel_rep_senate_1010_101412_314271,1,NRSC-MANDEL/REPUBLICAN
5,459790-nrsc_mandel_rep_senate_1010_101412_314271,of,NRSC-MANDEL/REPUBLICAN
6,459790-nrsc_mandel_rep_senate_1010_101412_314271,3,NRSC-MANDEL/REPUBLICAN
7,459790-nrsc_mandel_rep_senate_1010_101412_314271,WCPO,NRSC-MANDEL/REPUBLICAN
8,459790-nrsc_mandel_rep_senate_1010_101412_314271,ORDER,NRSC-MANDEL/REPUBLICAN
9,459790-nrsc_mandel_rep_senate_1010_101412_314271,Flight,NRSC-MANDEL/REPUBLICAN


In [48]:
df_all['slug'][0]

'459790-nrsc_mandel_rep_senate_1010_101412_314271'

In [49]:
df_group = df_all.groupby(['slug','committee'])['token'].apply(lambda a: ' '.join([str(x) for x in a])).reset_index()
df_group.head(50)

,slug,committee,token
0,416903-collect-files-39738-political-file-2012...,60+ ASSOCIATION,Print Date 8/14/12 Page 1 of 1 WXIX ORDER Flig...
1,416904-collect-files-39738-political-file-2012...,60+ ASSOCIATION,Print Date 8/14/12 Page 1 of 1 WXIX ORDER Flig...
2,416945-collect-files-59441-political-file-2012...,60+ ASSOCIATION,Print Date 8/17/12 Page 1 of 2 WEWS ORDER Flig...
3,416946-collect-files-59441-political-file-2012...,60+ ASSOCIATION,Print Date 8/17/12 Page 1 of 2 WEWS ORDER Flig...
4,419754-collect-files-65684-political-file-2012...,WARREN/D/SENATE,Contract Agreement Between: Print Date 8/23/12...
5,419761-collect-files-65684-political-file-2012...,WARREN/D/SENATE,Contract Agreement Between: Print Date 8/29/12...
6,420232-collect-files-39746-political-file-2012...,60+ ASSOCIATION,Contract Agreement Between: Print Date 8/14/12...
7,420233-collect-files-39746-political-file-2012...,60+ ASSOCIATION,Contract Agreement Between: Print Date 8/14/12...
8,420426-collect-files-72313-political-file-2012...,PRIORITIES USA ACTION,Print Date 8/13/12 16:34:44 Page 1 of 2 ORDER ...
9,424793-collect-files-39738-political-file-2012...,60+ ASSOCIATION,Print Date 9/6/12 Page 1 of 1 WXIX ORDER Fligh...


In [50]:
print(df_group['slug'][0])

' '.join(df_all[df_all['slug'] == '416903-collect-files-39738-political-file-2012-non']['token'])

416903-collect-files-39738-political-file-2012-non


'Print Date 8/14/12 Page 1 of 1 WXIX ORDER Flight Dates08/15/12-08/21/12 Order Sep 0:00:00 Product Estimate # 1224 Contract / Revision 808557 / 60 PLUS ASSOC SENATE Alt Order # 6276486 Original Date / Revision Agency Com 15% Billing Type Cash 8/14/12 8/14/12 Order Type Political Billing Contact Billing Cycle EOM/EOC Advertiser60 Plus Assoc 600 Farimont Avenue Billing CalendarBROADCAST Suite 306 Demographic A35+ Towson, MD 21286 Agency Mentzer Media Services Rev Codes Agency Political Pol-Issue Buying Contact Sales Office T-DC Product Codes PL20 Sales Region National Priority P 1 600 Farimont Avenue Suite 306 Agency Ref Advertiser Ref Towson, MD 21286 Primary Account Executive Aaron Ashe Account Executive Order% Start Date End Date Aaron Ashe 100% Order Share % Market Value Competing Station % of Order Amount EXIX % WBQC % WCET % WCPO % WCVN % WKON % WKRC % WLWT % WPTO % WSTR % Order Totals Billing Plan Month # of Spots Net Amount Gross Amount Rating Start Date End Date # Spots Net Amou

# Applying this method to our data: 

In [51]:
batch_size = 32  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.
# Path to the data txt file on disk.

In [52]:
input_texts = []
input_texts = df_group['token'][:num_samples]

input_characters = set()
for token in input_texts:
        for char in token:
            if char not in input_characters:
                input_characters.add(char)
            if ' ' not in input_characters: 
                input_characters.add(' ')
        if '\t' not in input_characters: 
            input_characters.add('\t')   

In [53]:
target_texts = []
target_texts = df_group['committee'][:num_samples]

target_characters = set()
for token in target_texts:
    if token != 'nan':
        for char in token:
            if char not in target_characters:
                target_characters.add(char)
        if ' ' not in target_characters: 
            target_characters.add(' ')
        if '\t' not in target_characters: 
            target_characters.add('\t')        
        


In [54]:
input_characters = list(input_characters)
target_characters = list(target_characters)
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

#print(input_texts[:10])
#print(input_characters)

Number of unique input tokens: 80
Number of unique output tokens: 26
Max sequence length for inputs: 17431
Max sequence length for outputs: 35


In [55]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
print (input_token_index)
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

{'S': 0, 'D': 1, '9': 2, 'L': 3, '=': 4, 'k': 5, '1': 6, 'v': 7, '(': 8, '$': 9, 'c': 10, 'e': 11, 'A': 12, '.': 13, '*': 14, 'j': 15, 'g': 16, 'I': 17, 'm': 18, 'o': 19, 'G': 20, 'N': 21, 'a': 22, '6': 23, 'C': 24, 'h': 25, '4': 26, 'P': 27, 'M': 28, 'O': 29, '3': 30, '-': 31, 'u': 32, 'x': 33, 'r': 34, 'W': 35, '0': 36, '5': 37, 'J': 38, 'F': 39, 'Z': 40, '%': 41, '/': 42, '#': 43, '?': 44, ',': 45, ')': 46, '7': 47, 'R': 48, 'B': 49, ' ': 50, ':': 51, 'd': 52, 't': 53, 'l': 54, '2': 55, 'H': 56, 'n': 57, 'w': 58, 'T': 59, '+': 60, 'q': 61, '8': 62, '@': 63, 'z': 64, 'Q': 65, '&': 66, 'i': 67, '\t': 68, 'y': 69, 'b': 70, 'f': 71, 'V': 72, 'E': 73, 's': 74, 'X': 75, 'K': 76, 'Y': 77, 'p': 78, 'U': 79}


In [56]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        #decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [57]:
cnnVendorRead = models.Sequential()
# Layer 1 = input layer
# specify the input size in the first layer.
#------------------------------------------
# Layer 1 = input layer
cnnVendorRead.add(layers.Conv1D(32, 3, activation="relu", input_shape=B.shape[1:3] ))

cnnVendorRead.add(layers.MaxPooling1D((2)))


# Layer 2
cnnVendorRead.add(layers.Conv1D(64, 3, activation="relu"))

cnnVendorRead.add(layers.MaxPooling1D((2)))


# Layer 3
cnnVendorRead.add(layers.Conv1D(64, 3, activation="relu" ))

cnnVendorRead.add(layers.Flatten())
#------------------------------------------

#dnnModel.add(layers.Dense(64, activation='elu', input_shape= (10,20,)))
cnnVendorRead.add(layers.Dense(20, activation='relu'))

# Layer 2 = hidden layer 
#dnnModel.add(layers.Dense(60, activation='relu'))

# Layer 3 = hidden layer 
#dnnModel.add(layers.Dense(30, activation='linear'))

# Layer 4 = output layer
cnnVendorRead.add(layers.Dense(len(category2idx), activation='softmax'))

cnnVendorRead.summary()
cnnVendorRead.compile( optimizer = 'adam', loss = 'cosine_similarity', metrics=['accuracy'] )

In [58]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [59]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)



In [60]:
print(decoder_outputs)

Tensor("dense_2/Identity:0", shape=(None, None, 26), dtype=float32)


In [ ]:
cnnVendorRead.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 3.2797 - accuracy: 0.0026 - val_loss: 3.0784 - val_accuracy: 0.3905
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 2.9236 - accuracy: 0.6130 - val_loss: 2.7438 - val_accuracy: 0.3905
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 1.4815 - accuracy: 0.6104 - val_loss: 3.0319 - val_accuracy: 0.2952
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: 2.7606 - accuracy: 0.5013 - val_loss: 2.6287 - val_accuracy: 0.4095
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 2.0887 - accuracy: 0.6545 - val_loss: 3.3461 - val_accuracy: 0.4000
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 1.3776 - accuracy: 0.6338 - val_loss: 2.6757 - val_accuracy: 0.4190
Epoch 7/10
1/1 [==============================] - 2s 2s/step - loss: 1.4758 - accuracy: 0.6753 - val_loss: 3.5432 - val_accuracy: 0.3905
Epoch 8/10
1/1 [=========================

In [42]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Print Date 8/14/12 Page 1 of 1 WXIX ORDER Flight Dates08/15/12-08/21/12 Order Sep 0:00:00 Product Estimate # 1224 Contract / Revision 808557 / 60 PLUS ASSOC SENATE Alt Order # 6276486 Original Date / Revision Agency Com 15% Billing Type Cash 8/14/12 8/14/12 Order Type Political Billing Contact Billing Cycle EOM/EOC Advertiser60 Plus Assoc 600 Farimont Avenue Billing CalendarBROADCAST Suite 306 Demographic A35+ Towson, MD 21286 Agency Mentzer Media Services Rev Codes Agency Political Pol-Issue Buying Contact Sales Office T-DC Product Codes PL20 Sales Region National Priority P 1 600 Farimont Avenue Suite 306 Agency Ref Advertiser Ref Towson, MD 21286 Primary Account Executive Aaron Ashe Account Executive Order% Start Date End Date Aaron Ashe 100% Order Share % Market Value Competing Station % of Order Amount EXIX % WBQC % WCET % WCPO % WCVN % WKON % WKRC % WLWT % WPTO % WSTR % Order Totals Billing Plan Month # of Spots Net Amount Gross Amount Rating Start Date End Date

-
Input sentence: Contract Agreement Between: Print Date 8/23/12 Page 1 of 6 CONTRACT WCVB Contract / Revision Alt Order # 5 TV Place 944723 / Needham, MA 2492 Product (781)449-0400 8/27-9/2 Contract Dates Estimate # www.thebostonchannel.com 8/27/12 - 9/2/12 11662 Advertiser Original Date / Revision Warren/D/Senate 8/22/12 / 8/23/12 And: Billing Cycle Billing Calendar Cash/Trade EOM/EOC Broadcast Cash Media Strategies - VA Station Account Executive Sales Office 9990 Lee Highway/#210 Fairfax, VA 22030 WCVB Scott Tarka Boston Special Handling Demographic Adults 25-54 IDB# Advertiser Code Product Code Agency Ref Advertiser Ref Spots/ *Line Ch Start DateEnd Date Description Start/End Time Days Length Week Rate TypeSpots Amount N 1 WCVB08/27/12 9/2/12 5a Eyeopener 5-6a :30 NM 10 $3,500.00  Class of Time - Immediately Pre-emptible without notice Start Date End Date Weekdays Spots/Week Rate Week: 8/27/12 9/2/12 MTWTF-- 10 $350.00  N 2 WCVB08/27/12 9/2/12 6a Eyeopener 558-7a :30 NM 10 $6,500.0

-
Input sentence: Contract Agreement Between: Print Date 8/29/12 Page 1 of 7 CONTRACT WCVB Contract / Revision Alt Order # 5 TV Place 940943 / Needham, MA 2492 Product (781)449-0400 8/20-8/26 Contract Dates Estimate # www.thebostonchannel.com 8/20/12 - 8/26/12 11586 Advertiser Original Date / Revision Warren/D/Senate 8/9/12 / 8/29/12 And: Billing Cycle Billing Calendar Cash/Trade EOM/EOC Broadcast Cash Media Strategies - VA Station Account Executive Sales Office 9990 Lee Highway/#210 Fairfax, VA 22030 WCVB Scott Tarka Boston Special Handling Demographic Adults 25-54 IDB# Advertiser Code Product Code Agency Ref Advertiser Ref Spots/ *Line Ch Start DateEnd Date Description Start/End Time Days Length Week Rate TypeSpots Amount N 1 WCVB08/20/12 8/26/12 5a Eyeopener 5-6a :30 NM 10 $3,500.00  Class of Time - Immediately Pre-emptible without notice - makegood likely Start Date End Date Weekdays Spots/Week Rate Week: 8/20/12 8/26/12 MTWTF-- 10 $350.00  N 2 WCVB08/20/12 8/26/12 6a Eyeopener 558

-
Input sentence: Contract Agreement Between: Print Date 8/14/12 Page 1 of 1 CONTRACT WOIO Contract / Revision Alt Order # 1717 East 12th Street 808532 / 6276501 Cleveland, OH 44114 Product -216 771-1943 60 PLUS ASSOC SENATE Contract Dates Estimate # 8/15/12 - 8/21/12 1223 Advertiser Original Date / Revision 60 Plus Assoc 8/14/12 / 8/14/12 And: Billing Cycle Billing Calendar Cash/Trade EOM/EOC Broadcast Cash Mentzer Media Services Station Account Executive Sales Office 600 Farimont Avenue Suite 306 WOIO Teresa Difuria Telerep-Philadelphia Towson, MD 21286 Special Handling Demographic Adults 35+ IDB# Advertiser Code Product Code 1021 Agency Ref Advertiser Ref Spots/ Totals *Line Ch Start DateEnd Date Description Start/End Time Days Length Week Rate TypeSpots Amount N 1 WOIO 8/15/12 8/17/12 Action News at 11p 11-1135p :30 NM 2 $2,000.00  Start Date End Date Weekdays Spots/Week Rate Week: 8/13/12 8/19/12 --W-F-- 2 $1,000.00  N 2 WOIO 8/19/12 8/19/12 Face the Nation 1030-1130a :30 NM 1 $45

-
Input sentence: Print Date 9/6/12 Page 1 of 1 WXIX ORDER Flight Dates08/22/12-08/28/12 Order Sep 0:00:00 Product Estimate # 1225 Contract / Revision 808561 / 60 PLUS ASSOC SENATE Alt Order # 6276490 Original Date / Revision Agency Com 15% Billing Type Cash 8/14/12 8/14/12 Order Type Political Billing Contact Billing Cycle EOM/EOC Advertiser60 Plus Assoc 600 Farimont Avenue Billing CalendarBROADCAST Suite 306 Demographic A35+ Towson, MD 21286 Agency Mentzer Media Services Rev Codes Agency Political Pol-Issue Buying Contact Sales Office T-DC Product Codes PL20 Sales Region National Priority P 1 600 Farimont Avenue Suite 306 Agency Ref Advertiser Ref Towson, MD 21286 Primary Account Executive Aaron Ashe Account Executive Order% Start Date End Date Aaron Ashe 100% Order Share % Market Value Competing Station % of Order Amount EXIX % WBQC % WCET % WCPO % WCVN % WKON % WKRC % WLWT % WPTO % WSTR % Order Totals Billing Plan Month # of Spots Net Amount Gross Amount Rating Start Date End Date 

-
Input sentence: Contract Agreement Between: Print Date 8/6/12 Page 1 of 2 CONTRACT WJBK Contract / Revision Alt Order # 16550 West 9 Mile Road 116252 / Shanelle Jackson - State 13th District Rep Southfield, MI 48075 Product -248 557-2000 August 2012 Primary Contract Dates Estimate # 8/6/12 - 8/7/12 August 2012 Primary Advertiser Original Date / Revision Committee to Elect Shanelle Jackson 8/3/12 / 8/3/12 And: Billing Cycle Billing Calendar Cash/Trade EOM/EOC Broadcast Cash Shanelle Jackson for Congress Station Account Executive Sales Office Attention: Shanelle Jackson 9730 W. Outer Drive WJBK Matt Joseph Detroit-Local #102 Special Handling Detriot, MI 48223 Demographic Adults 35+ IDB# Advertiser Code Product Code Agency Ref Advertiser Ref Spots/ *Line Ch Start DateEnd Date Description Start/End Time Days Length Week Rate TypeSpots Amount 1 WJBK 8/6/12 8/7/12 Overnight 2a-3a :30 NM 2 $70.00  Candidate Discount Start Date End Date Weekdays Spots/Week Rate Week: 8/6/12 8/12/12 M------ 2

UnboundLocalError: local variable 'batch_outputs' referenced before assignment